# 1. Data collection

In [4]:
import torch
print(torch.__version__)

1.0.0a0+18eef1d


In [10]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [5]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

Directories not created becasue they already exist


In [6]:
button_layout = widgets.Layout(width='128px', height='64px')
free_button = widgets.Button(description='add free', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='add blocked', button_style='danger', layout=button_layout)
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
# attach the callbacks, we use a 'lambda' function to ignore the
# parameter that the on_click event would provide to our function
# because we don't need it.
free_button.on_click(lambda x: save_free())
blocked_button.on_click(lambda x: save_blocked())

In [5]:
display(image)
display(widgets.HBox([free_count, free_button]))
display(widgets.HBox([blocked_count, blocked_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
!zip -r -q dataset.zip dataset

# 2. Train model

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
!unzip -q dataset.zip

In [3]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [4]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [6]:
model = models.alexnet(pretrained=True)

In [7]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [8]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

In [3]:
pip install keras

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.7 MB 5.1 MB/s eta 0:00:01     |█████████████████████████       | 1.3 MB 5.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


# 3. Buzzer control

In [1]:
import os
os.system("sudo chmod 777 /dev/ttyTHS1")

256

In [2]:
import RPi.GPIO as GPIO

BEEP_pin = 6 # buzzer pin number

GPIO.setmode(GPIO.BCM)
GPIO.setup(BEEP_pin, GPIO.OUT, initial = GPIO.LOW) # starting option

/usr/local/lib/python3.6/dist-packages/Jetson/GPIO/gpio.py:352: RuntimeWarning: This channel is already in use, continuing anyway. Use GPIO.setwarnings(False) to disable warnings
  RuntimeWarning)


In [3]:
import time

def Beep(beeptime):
    GPIO.output(BEEP_pin, GPIO.HIGH) # buzzer on
    time.sleep(beeptime) # sound time
    GPIO.output(BEEP_pin, GPIO.LOW) # buzzer off

In [4]:
Beep(0.3)

# 4. Automatic avoid

In [5]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [2]:
model.load_state_dict(torch.load('best_model.pth'))

In [3]:
device = torch.device('cuda')
model = model.to(device)

In [4]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224,fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

In [8]:
from jetbot import Robot

robot = Robot()

In [14]:
import torch.nn.functional as F
import time

def update(change):
#     global blocked_slider, robot
#     x = change['new'] 
#     x = preprocess(x)
#     y = model(x)
    
#     # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
#     y = F.softmax(y, dim=1)
    
#     prob_blocked = float(y.flatten()[0])
    
#     blocked_slider.value = prob_blocked
    prob_blocked=0.8
    
    
    if prob_blocked > 0.7 :
        robot.stop()
        Beep(1)

        robot.left(1)
        time.sleep(0.27)
        robot.stop()
        robot.forward(2)
        time.sleep(0.5)
        robot.stop()
        robot.right(1)
        time.sleep(0.)
        robot.stop()
    else :
        robot.stop()
        robot.forward(2)
        time.sleep(0.5)
        robot.stop()
        return 1
        
#update({'new': camera.value})  # we call the function once to intialize

In [17]:
i=0
robot.forward(2)
while(i<2) :
#     if update({'new': camera.value}) == 0:
#     print('robot blocked.')
#     else :
#         print('automatic avoided.')
#         break
    update(0)
    robot.forward(2)
    time.sleep(0.5)
    i += 1
robot.stop() 